In [1]:
import xarray as xr
import rasterio

In [3]:
ds = xr.open_dataset(r"C:\Users\Lewis\Desktop\stac-6cd53a2cb53a323c4bbd061f6313b880.nc")

In [6]:
ds.geobox

AttributeError: 'Dataset' object has no attribute 'geobox'